In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import *

In [2]:
df_business_a = pd.read_parquet("./data/business_attributes.parquet")

df_review = pd.read_parquet("./data/review.parquet")

df_tip = pd.read_parquet("./data/tip.parquet")

In [4]:
df_business = pd.read_parquet("./data/business.parquet")
df_business_h = pd.read_parquet("./data/business_hours.parquet")

elements = []
for key, value in zip(df_business_h["day"], df_business_h["open_close"]):
    open_hour, close_hour = value.split("-")
    dt_open = datetime.strptime(open_hour, "%H:%M")
    dt_close = datetime.strptime(close_hour, "%H:%M")
    elements.append({key: tuple((dt_open.time(), dt_close.time()))})

s = pd.Series(elements)

df_b = pd.concat([df_business_h['business_id'], s], axis=1)
df_b = df_b.rename(columns = {0: "working_days"})

df_b["working_days"] = df_b["working_days"].apply(lambda x: dict(x))

df_b = (
    df_b.groupby("business_id", sort=False, as_index=False)
        .agg({"working_days": lambda lst: {k: v for d in lst for k, v in d.items()}})
)

df_merged = pd.merge(df_business, df_b, on="business_id", how="left")

df_merged.to_parquet("./data/business_hours_merge.parquet")